# Networkx 

[Networkx basic tutorial](http://pynetwork.readthedocs.io/en/latest/networkx_basics.html)

In [ ]:
%matplotlib inline
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import itertools
import operator

### Preferential wiring

Many real (world) networks have degree distributions that look like:  

__Power Law = (𝑃𝑘 = C𝑘^α) (typical α ± 2-4)__

Degree distribution: pdf/relative frequency of the (in-)degrees over the entire network.

__Preferential Attachment Model__ produces networks with a power law degree distribution:

Probability of connecting to a node μ of degree k_μ is (k_μ / sum(k_γ))
Probability of node is relative degree: degrees_node/total_degree_graph)

Each new node is attached according to this probability distribution.

#### Barabasi-Albert

barabasi_albert_graph(n, m):
 - n-node preferential attachment network, where:
 - each new node attaches to m existing nodes

In [ ]:
G = nx.barabasi_albert_graph(100000, 1)
degrees = G.degree()
dict_degrees = {k: v for k, v in sorted(degrees)}
degree_set = sorted(set(dict_degrees.values()))
degree_list = list(dict_degrees.values())

In [ ]:
histogram = [degree_list.count(i)/float(nx.number_of_nodes(G)) for i in degree_set]
plt.plot(degree_set, histogram, 'o')
plt.xlabel('Degree')
plt.ylabel('Fraction of Nodes')
plt.show();

#### Log scale

In [ ]:
histogram = [degree_list.count(i)/float(nx.number_of_nodes(G)) for i in degree_set]
plt.plot(degree_set, histogram, 'o')
plt.xlabel('Degree')
plt.ylabel('Fraction of Nodes')
plt.xscale('log')
plt.yscale('log')
plt.show();

### Rewiring

Social networks tend to have high clustering coefficient and small average path length.

##### Average Clustering Coefficient(Average CC)
Average clustering of entire network by averaging all local clustering coefficient values of all nodes.

##### Local Clustering Coefficient(Local CC): 
Number of pairs of A’s friends who are friends with each other / # all possible pairs of A’s friends

##### Average shortest path length - n-degrees of separation
Shortest path length is the shortest distance from a start node to the end node.

##### median path length: typically between 4-7
TODO


In [ ]:
G = nx.barabasi_albert_graph(1000, 6)

In [ ]:
nx.average_clustering(G)

In [ ]:
nx.average_shortest_path_length(G)

### Path length distribution

Motivation:  
Real networks exhibit high clustering coefficient and small average shortest paths.
Can we think of a model that achieves both of these properties?

a. Regular Lattice (𝑝 = 0): no edge is rewired  
b. Random Network (𝑝 = 1): all edges are rewired  
c. Small World Network (0 < 𝑝 < 1): Some edges are rewired. Network conserves some local structure but has some randomness  

### Watts-Strogatz

watts_strogatz_graph(n, k, p) returns a small world network with n nodes starting with a ring lattice with each node connected to its k nearest neighbors, rewiring probability p.

In [ ]:
G = nx.watts_strogatz_graph(1000, 6, 0.04)
degrees = G.degree()

In [ ]:
dict_degrees = {k: v for k, v in sorted(degrees)}
degree_set = sorted(set(dict_degrees.values()))
degree_list = list(dict_degrees.values())

In [ ]:
histogram = [degree_list.count(i)/float(nx.number_of_nodes(G)) for i in degree_set]
plt.bar(degree_set, histogram)
plt.xlabel('Degree')
plt.ylabel('Fraction of Nodes')
plt.show();

In [ ]:
# No power law degree distribution:
#   Since most edges are not rewired, most nodes have degree of 6.
#   Since edges are rewired uniformly at random, no node accumulated very high degree, like in the preferential attachment model
plt.plot(degree_set, histogram, 'o')
plt.xlabel('Degree')
plt.ylabel('Fraction of Nodes')
plt.xscale('log')
plt.yscale('log')
plt.show();

In [ ]:
G = nx.connected_watts_strogatz_graph(n, k, p, t)
#  runs watts_strogatz_graph(n, k, p) up to t times, until it returns a connected small world network

In [ ]:
G = nx.newman_watts_strogatz_graph(n, k, p)
# runs a model similar to the small world model, but rather than rewiring edges, new edges are added with probability 𝑝

### Summary:

- __Real social networks__:
 - small shortest paths
 - high clustering coefficient
- __Preferential attachment model__:
 - small shortest paths
 - very small clustering
- __Small world model__(p = small):
 - small average shortest path
 - high clustering coefficient (=matching real networks). However, the degree distribution is not a power law

watts_strogatz_graph(n, k, p) (and other variants) to produce small world networks

### Link Prediction

Given a network, can we predict which edges will be formed in the future?
- What new edges are likely to form in this network?
- Given a pair of nodes, how to assess whether they are likely to connect?

__Triadic closure__:  
The tendency for people who share connections in a social network to become connected.

#### Measure 1: Common Neighbors (intercept)
The number of common neighbors of nodes 𝑋 and 𝑌

In [ ]:
G = nx.newman_watts_strogatz_graph(100, 5, 0.1)
common_neigh = [(e[0], e[1], len(list(nx.common_neighbors(G, e[0], e[1])))) for e in nx.non_edges(G)]
sorted(common_neigh, key=operator.itemgetter(2), reverse=True)[:5]

#### Jaccard Coefficient (intercept over union)
Number of common neighbors normalized by the total number of neighbors
common_neighbors/total_neighbors

In [ ]:
L = list(nx.jaccard_coefficient(G))
L.sort(key=operator.itemgetter(2), reverse=True)
L[:5]

#### Resource
Fraction of a ”resource” that a node can send to another through their common neighbors
sum(1/degree_common_neighbor)

In [ ]:
L = list(nx.resource_allocation_index(G))
L.sort(key=operator.itemgetter(2), reverse=True)
L[:5]

#### Adamic Adar Index
Similar to resource allocation index, but with log in the denominator
sum(1/log(degree_common_neighbor))

In [ ]:
L = list(nx.adamic_adar_index(G))
L.sort(key=operator.itemgetter(2), reverse=True)
L[:5]

#### Preferential Attachment
In the preferential attachment model, nodes with high degree get more neighbors
degree_source * degree_target

In [ ]:
L = list(nx.preferential_attachment(G))
L.sort(key=operator.itemgetter(2), reverse=True)
L[:5]

#### Community Common Neighbors
Number of common neighbors with bonus of 1 for each neighbor in same community
f(u) = 1 if same community else 0
sum(f(u) * degree)

In [ ]:
G = nx.newman_watts_strogatz_graph(9, 5, 0.1)
G.nodes()
G.node[0]['community'] = 0
G.node[1]['community'] = 0
G.node[2]['community'] = 0
G.node[3]['community'] = 0
G.node[4]['community'] = 1
G.node[5]['community'] = 1
G.node[6]['community'] = 1
G.node[7]['community'] = 1
G.node[8]['community'] = 1
L = list(nx.cn_soundarajan_hopcroft(G))
L.sort(key=operator.itemgetter(2), reverse=True); L

#### Community Resource Allocation
Similar to resource allocation index, but only considering nodes in the same community  
f(u) = 1 if same community else 0  
sum(f(u)/degree)

In [ ]:
L = list(nx.ra_index_soundarajan_hopcroft(G))
L.sort(key=operator.itemgetter(2), reverse=True)
L[:5]

### Summary
Link prediction problem:  
Given a network, predict which edges will be formed in the future.  

5 basic measures:  
– NumberofCommonNeighbors – JaccardCoefficient  
– ResourceAllocationIndex  
– Adamic-AdarIndex  
– PreferentialAttachmentScore  

2 measures that require community information:  
– CommonNeighbor-Soundarajan-HopcroftScore  
– ResourceAllocation-Soundarajan-HopcroftScore  

### Plot

In [ ]:
# See what layouts are available in networkX
[x for x in nx.__dir__() if x.endswith('_layout')]

In [ ]:
# draw the graph using the default spring layout
plt.figure(figsize=(12, 8))

pos_a = nx.random_layout(G)                   # for nodes without geo location attribute
pos_b = nx.get_node_attributes(G, 'location') # for nodes with geo location attribute

nx.draw_networkx(G, pos_a, alpha=0.7, with_labels=False, edge_color='.4')
plt.axis('off')
plt.show;

Draw graph with varying node color, node size, and edge width

In [ ]:
plt.figure(figsize=(12, 8))
pos_a = nx.fruchterman_reingold_layout(G)

node_color =  [1000*nx.degree_centrality(G)[v] for v in G] # [50*nx.degree(G)[v] for v in G]
node_size = [100*G.degree(v) for v in G]
edge_width = [20*nx.betweenness_centrality(G, normalized=True, endpoints=False)[v] for v in G]

# Option when weighted edges
# edge_width = [0.0015*G[u][v]['weight'] for u, v in G.edges()]

nx.draw_networkx(G, 
                 pos_a, 
                 node_size=node_size,
                 node_color=node_color, 
                 alpha=0.7, 
                 with_labels=True,
                 width=edge_width, 
                 edge_color='.4', 
                 cmap=plt.cm.Blues)

# nx.draw_networkx_labels(G, pos_a, labels={'1': '1', '2': '2'}, font_size=18, font_color='w')

plt.axis('off')
plt.tight_layout();